### Import PySpark
Nogle få kommentarer:
 - Download spark pre-built for hadoop 2.6, jeg vil også anbefale jer at bruge spark 1.6.0 da der er nogle problemer med 1.6.1 [hent den her http://www.apache.org/dyn/closer.lua/spark/spark-1.6.0/spark-1.6.0-bin-hadoop2.6.tgz]
 - husk at ændre paths i denne notebook
 - `os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.databricks:spark-avro_2.10:2.0.1 pyspark-shell"` vil give jer mulighed for direkte at loade avro filer

In [1]:
import sys
import os
import os.path

SPARK_HOME = """spark-1.6.0-bin-hadoop2.6/""" ## PATH TO SPARK

sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "py4j-0.9-src.zip"))
sys.path.append(os.path.join(SPARK_HOME, "python", "lib", "pyspark.zip"))
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.databricks:spark-avro_2.10:2.0.1 pyspark-shell"
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"


from pyspark import SparkConf, SparkContext, StorageLevel
from pyspark.sql import SQLContext

conf = (SparkConf()
         .setMaster("local[*]")
         .setAppName("My app"))
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [2]:
# load data to dataframe
df = (sqlContext.read.format("com.databricks.spark.avro")
      .load("data/201512/*.avro")
      )

In [3]:
# I kan se strukturen på data her
df.printSchema()

root
 |-- timestamp_seen: long (nullable = false)
 |-- id: string (nullable = false)
 |-- useruuid: string (nullable = false)
 |-- start_time: string (nullable = false)
 |-- end_time: string (nullable = false)
 |-- name: string (nullable = false)
 |-- area: string (nullable = false)
 |-- country: string (nullable = false)
 |-- region: string (nullable = false)
 |-- latitude: double (nullable = false)
 |-- longitude: double (nullable = false)
 |-- altitude: integer (nullable = false)
 |-- accuracy: integer (nullable = false)
 |-- datum: string (nullable = false)
 |-- devices: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- name: string (nullable = false)
 |    |    |-- type: string (nullable = false)
 |    |    |-- id: string (nullable = false)



In [4]:
# kig på data
df.show(10)

+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------+------+-----------------+------------------+--------+--------+-----+--------------------+
|timestamp_seen|                  id|            useruuid|          start_time|            end_time|                name|         area|country|region|         latitude|         longitude|altitude|accuracy|datum|             devices|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-------+------+-----------------+------------------+--------+--------+-----+--------------------+
| 1449555713263|fd16a2d9-810a-43c...|b81264f3-511d-45a...|2015-12-08T07:03:...|2015-12-08T07:07:...|                Bjuv|        Skane| Sweden|Europe|56.10314760312085|12.866914578182298|       0|   26000|WGS84|[[E5823,PHONE,ad6...|
| 1449555723641|1129abd4-cadc-449...|694237c5-cd52-45c...|2015-12-08

In [5]:
# transform til rdd
data = df.rdd

In [6]:
data.count()

687718

In [7]:
#more imports
from DatabaseHelper import DatabaseHelper
from dateutil import parser

db = DatabaseHelper()

first_period_min_date = parser.parse("2015-12-01 00:00:00+00:00")
first_period_max_date = parser.parse("2015-12-31 23:59:59+00:00")
second_period_min_date = parser.parse("2016-01-01 00:00:00+00:00")
second_period_max_date = parser.parse("2016-01-31 23:59:59+00:00")
third_period_min_date = parser.parse("2016-02-01 00:00:00+00:00")
third_period_max_date = parser.parse("2016-02-29 23:59:59+00:00")


In [44]:
#filtering
swe_data = data.filter(lambda row: "Sweden" in row["country"]).filter(lambda row: parser.parse(row["start_time"]) >= first_period_min_date
                           and parser.parse(row["end_time"]) <= third_period_max_date)

# convert to ((spatial, time)[useruuid]) rows
def convert_time_and_spatial(row):
    return [((db.calculate_spatial_bin(row["longitude"], row["latitude"]),x),[row["useruuid"]]) for x in db.calculate_time_bins(row["start_time"], row["end_time"])]


binned_swe_data = swe_data.flatMap(convert_time_and_spatial)

# remove duplicates and reduce to (spatial_bin, time_bin) -> [users]
bins_to_users = binned_swe_data.reduceByKey(lambda a, b: a+b if b[0] not in a else a)

In [46]:
# divide into periods
first_period_min_bin = db.calculate_time_bins("2015-12-01 00:00:00+00:00")[0]
first_period_max_bin = db.calculate_time_bins("2015-12-09 23:59:59+00:00")[0]
second_period_min_bin = db.calculate_time_bins("2015-12-10 00:00:00+00:00")[0]
second_period_max_bin = db.calculate_time_bins("2015-12-19 23:59:59+00:00")[0]
third_period_min_bin = db.calculate_time_bins("2015-12-20 00:00:00+00:00")[0]
third_period_max_bin = db.calculate_time_bins("2015-12-31 23:59:59+00:00")[0]

# remove duplicates and create (users) -> [(spatial_bin, time_bin)]
period_1_bins_to_users = bins_to_users.filter(lambda row: row[0][1] >= first_period_min_bin
                                and row[0][1] < first_period_max_bin)
period_1_bins_to_users.map(lambda row: ((row[1][0]),[row[0]])).reduceByKey(lambda a, b: a+b if b[0] not in a else a)
period_2_bins_to_users = bins_to_users.filter(lambda row: row[0][1] >= second_period_min_bin
                                and row[0][1] < second_period_max_bin)
period_2_users_to_bins = period_2_bins_to_users.map(lambda row: ((row[1][0]),[row[0]])).reduceByKey(lambda a, b: a+b if b[0] not in a else a)
period_3_bins_to_users = bins_to_users.filter(lambda row: row[0][1] >= third_period_min_bin
                                and row[0][1] < third_period_max_bin)
period_3_users_to_bins = period_3_bins_to_users.map(lambda row: ((row[1][0]),[row[0]])).reduceByKey(lambda a, b: a+b if b[0] not in a else a)

In [10]:
print(period_1_data.count())
print(period_2_data.count())
print(period_3_data.count())

48387
62887
73800


In [11]:
from itertools import combinations
def generate_cooccurrences(row):
    return [(tuple(sorted(pair)),[row[0]]) for pair in combinations(row[1], 2)]
    
coocs_1 = period_1_data.flatMap(generate_cooccurrences).reduceByKey(lambda a,b: a+b)
coocs_2 = period_2_data.flatMap(generate_cooccurrences).reduceByKey(lambda a,b: a+b)
coocs_3 = period_3_data.flatMap(generate_cooccurrences).reduceByKey(lambda a,b: a+b)
                                                                    

In [14]:
coocs_1.take(5)

[(('20f92378-817e-4bd2-8f19-10b8e5c58c39',
   'ab65c6e5-5ee9-45d0-a352-a52c7ef9d9d6'),
  [(26229253228, 2897),
   (26208913016, 2922),
   (26209093024, 2913),
   (26208553019, 2757),
   (26229433226, 2771),
   (26229433226, 2771),
   (26229433226, 2770),
   (26229433226, 2772),
   (26208553019, 2921),
   (26229253227, 2945),
   (26229433225, 2889),
   (26229433225, 2889),
   (26229433226, 2939),
   (26229433226, 2895),
   (26229433226, 2938),
   (26229433225, 2897),
   (26229433225, 2776)]),
 (('9935ae0d-b9a1-405b-9509-9847cd072abf',
   'fd90f2c4-1284-4b8f-b5f6-df9c67e2d653'),
  [(26229433226, 2943),
   (26229433226, 2943),
   (26229433225, 2940),
   (26229433225, 2940),
   (26229433225, 2941),
   (26229433225, 2942),
   (26229433225, 2943),
   (26229433225, 2936)]),
 (('21c884ed-d9b4-481e-ac94-29cf77bfd344',
   '8afe6fcd-b9b2-43bc-b0fb-b016628ce462'),
  [(26229433226, 2820),
   (26229433226, 2823),
   (26229433226, 2747),
   (26229433226, 2747),
   (26229433226, 2747),
   (26229433226

In [42]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.tree import RandomForest, RandomForestModel
import numpy as np

y_1_users = coocs_2.map(lambda row: row[0]).collect()
y_2_users = coocs_3.map(lambda row: row[0]).collect()

def compute_weighted_frequency(row, location_data):
    spatial_bins = [r[0] for r in row[1]]
    wf_value = 0
    for sb in set(spatial_bins):
        users = location_data.filter(lambda row: r[0][0] == sb).flatMap(lambda row: row[1]).foreach()
        wf_value += spatial_bins.count(sb)*np.exp()

def compute_features(y, row, location_data):
    num_coocs = len(row[1])
    num_unique_coocs = len(set([r[0] for r in row[1]]))
    spatial_bins = [r[0] for r in row[1]]
    
    diversity = -np.sum([spatial_bins.count(sb)/len(spatial_bins)*np.log2(spatial_bins.count(sb)/len(spatial_bins)) for sb in set(spatial_bins)])
    
    weighted_frequency = compute_weighted_frequency(row, location_data)
    return LabeledPoint(y, [num_coocs, num_unique_coocs, diversity])

def compute_train_features(row):
    y = 1 if row[0] in y_1_users else 0
    return compute_features(y, row, period_1_data)
def compute_test_features(row):
    y = 1 if row[0] in y_2_users else 0
    return compute_features(y, row, period_2_data)

X_train_num_coocs = coocs_1.map(compute_train_features)

X_test_num_coocs = coocs_2.map(compute_test_features)
# more features can be computed with join on different features maybe?
#X.take(1)
#X_train = RowMatrix(X)

In [43]:
model = RandomForest.trainClassifier(X_train_num_coocs, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

predictions = model.predict(X_test_num_coocs)

predictions = model.predict(X_test_num_coocs.map(lambda x: x.features))
labelsAndPredictions = X_test_num_coocs.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda vp: (vp[0] - vp[1]) * (vp[0] - vp[1])).sum() /\
    float(X_test_num_coocs.count())
print("Test Mean Squared Error = {}".format(str(testMSE)))

Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.